#### matplotlib

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def show_matplotlib_image(bgrImage, row_size=8, col_size=4):
    img_dst = cv2.cvtColor(bgrImage, cv2.COLOR_BGR2RGB)
    plt.figure(figsize = (row_size,col_size))
    plt.axis('off')
    plt.imshow(img_dst)
    plt.show()

#### 한글출력

In [ ]:
import cv2
import numpy as np
from PIL import ImageFont, ImageDraw, Image

def input_korean(img_dst, input_text, pt, color, fontsize=20):
    my_font = ImageFont.truetype("font/gulim.ttc", fontsize)
    img_dst = Image.fromarray(img_dst)
    mydraw = ImageDraw.Draw(img_dst)
    mydraw.text((pt[0], pt[1]), input_text, font=my_font, fill=(color[0], color[1], color[2]))
    return np.array(img_dst)

### 기본 이미지 출력

In [ ]:
import cv2
import numpy as np

img_src = cv2.imread('images/turtle.png', cv2.IMREAD_COLOR)
height, width = img_src.shape[:2]

cv2.imshow("src",img_src)

cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
import cv2

img_src = cv2.imread('images/shape.png')
height,width = img_src.shape[:2]
####### 0. 이미지의 잡음 제거를 위해 blur를 적용
img_blur = cv2.GaussianBlur(img_src, (3,3), 0)
####### 1. 이미지를 1채널 Gray영상으로 변환
img_gray = cv2.cvtColor(img_blur,cv2.COLOR_BGR2GRAY)
####### 2. 이진영상으로 만들기(threshold 적용)
# ret, img_bin = cv2.threshold(img_gray, 210, 255, cv2.THRESH_BINARY)
# img_bin = cv2.bitwise_not(img_bin) # 영상의 배경이 흰색이면 반전

ret, img_bin = cv2.threshold(img_gray, 210, 255, cv2.THRESH_BINARY_INV)

####### 3. 이진화를 개선하기위한 모폴로지 연산을 수행
kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (7,7))
img_bin = cv2.morphologyEx(img_bin, cv2.MORPH_CLOSE, kernel, iterations=2)

####### 4. 외곽선 추출(contour)
contours, hierarchy = cv2.findContours(img_bin, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)


###### 5. 외곽선 그리기(if문 적용하여 원하는 것만)
for i,contour in enumerate(contours):
    if(cv2.contourArea(contour) > 10000):
        cv2.drawContours(img_src, [contour], 0, (0, 0, 255), 2)
        cv2.putText(img_src, f'{i}: {cv2.contourArea(contour)}', tuple(contours[i][0][0]), cv2.FONT_HERSHEY_COMPLEX, 0.7, (0, 255, 0), 1)
        cv2.imshow("src", img_src)
        # cv2.waitKey(0)
        
cv2.waitKey()
cv2.destroyAllWindows()

### 최소 사각형 그리기

In [ ]:
import cv2
import numpy as np

img_src = cv2.imread('images/shape_broken.png')
height,width = img_src.shape[:2]
####### 0. 이미지의 노이즈나 잡음 제거를 위해 blur를 적용 하고 3*3 커널을 사용하고 고정점은 0으로 셋팅함
img_blur = cv2.GaussianBlur(img_src, (3,3), 0)
####### 1. 이미지를 1채널 Gray영상으로 변환
img_gray = cv2.cvtColor(img_blur,cv2.COLOR_BGR2GRAY)
####### 2. 이진영상으로 만들기(threshold 적용)
# ret, img_bin = cv2.threshold(img_gray, 210, 255, cv2.THRESH_BINARY)
# img_bin = cv2.bitwise_not(img_bin) # 영상의 배경이 흰색이면 반전
# 속성으로 이미자, 임계값, 최대값, 결과이미지를 반환함
ret, img_bin = cv2.threshold(img_gray, 210, 255, cv2.THRESH_BINARY_INV)
# 모폴리지 변환은 침식과 팽창을 통해 픽셀의 값을 커널내부의 극대값으로 대체(팽창) 하거나 커널내부의 극솟값으로 대체(침식)하게됨
# 이를 통해 팽창은 어두운 영역이 주어들고 밝은 영역이 늘어나며, 침식은 밝은 영역이 줄어들고 어두운 영역이 늘어남
# 3. 이진화를 개선하기위한 모폴로지 연산을 수행함 여기서 열림은 침식연산후 팽창연산을 수행하여 작은 반점이 사라지며 객체의 크기 감소를 원래대로 복구함
# 닫힘은 팽창연산을 적용후 침식연산을 적용하여 객체내의 구멍이 사라지며 발생한 크기 증가를 원래대로 복구한다.
#그외 그래디언트(팽창, 침식활용하여 그래이스케일 이미지가 급격하게 변하는 곳에서 가장 높은 결과를 반환함)
# 탑햇: 입력이미지와 열림의 조합으로 국소적으로 밝았떤 부부들이 분리됨
# 블랙햇: 입력이미지와 닫힘의 조합으로 어두운 영역이 채워져 사라졌던 홀 등이 표시되어 분리됨
# 히트미스: 이미지의 전경이나 배경 픽셀의 패턴을 찾는데 사용하는 이진 형태로 커널의 값이 0,1의 값만 가지고 있어 모서리를 검출하는데 유용함함
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
img_bin = cv2.morphologyEx(img_bin, cv2.MORPH_CLOSE, kernel, iterations=2)

####### 4. 이지화 이미지에서 외곽선 추출(contour)
contours, hierarchy = cv2.findContours(img_bin, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)


###### 5. 외곽선 그리기(if문 적용하여 원하는 것만)
for i,contour in enumerate(contours):
    if (cv2.contourArea(contour) > 1000):
        ### 사각형 구하기
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(img_src,(x, y),(x+w,y+h),(0,255,0),1)

        ### 최소 사각형 구하기
        rect = cv2.minAreaRect(contour)
        box = cv2.boxPoints(rect)
        box = np.int0(box)
        cv2.drawContours(img_src, [box],0, (255,0,255), 1)

        cv2.putText(img_src, f'{i}: {cv2.contourArea(contour)}', \
                    (contour[0][0][0],contour[0][0][1]), cv2.FONT_HERSHEY_COMPLEX, 0.7, (0, 255, 0), 1)
        cv2.imshow("src", img_src)


cv2.waitKey()
cv2.destroyAllWindows()